In [22]:
import pandas as pd

In [23]:
df=pd.read_csv("Churn_Modelling.csv")
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [24]:
df=df.drop(["RowNumber","CustomerId","Surname"],axis=1)

In [25]:
from sklearn.preprocessing import LabelEncoder , OneHotEncoder

label_encoder_gender = LabelEncoder()
df["Gender"] = label_encoder_gender.fit_transform(df["Gender"])
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [26]:
# one hot encoding "Geography"

onehot_encoder_geo= OneHotEncoder(handle_unknown="ignore")
geo_encoded= onehot_encoder_geo.fit_transform(df[["Geography"]]).toarray()
encoded_dataframe = pd .DataFrame(geo_encoded, columns=onehot_encoder_geo.get_feature_names_out(["Geography"]))

encoded_dataframe

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [27]:
df=pd.concat([df.drop("Geography",axis=1),encoded_dataframe],axis=1)
df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [28]:
x = df.drop(['EstimatedSalary'],axis=1)
y = df["EstimatedSalary"]

In [29]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [30]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train_Scaled = scaler.fit_transform(x_train)
x_test_Scaled = scaler.transform(x_test)

In [31]:
##save models
import pickle
with open("label_encoder_gender.pkl","wb") as file:
    pickle.dump(label_encoder_gender,file)
    
with open("onehot_encoder_geo.pkl","wb") as file:
    pickle.dump(onehot_encoder_geo,file)

with open("scaler.pkl","wb") as file:
    pickle.dump(scaler,file)

In [33]:
#first method

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

#build the model
model=Sequential(
    [
        Dense(64,activation="relu",input_dim=(x_train_Scaled.shape[1])),
        Dense(32,activation="relu"),
        Dense(1)
    ]
)
model.compile(loss="mean_absolute_error",optimizer="adam",metrics=["mae"])
model.summary()

c:\Users\himek\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
#second method


# from tensorflow.keras.models import Sequential 
# from tensorflow.keras.layers import Dense,Dropout

# #defining the ann model
# model=Sequential()

# #input layer
# model.add(Dense(68,activation="relu",input_dim=x_train_scaled.shape[1]))

# #hidden layer
# model.add(Dense(32,activation="relu"))
# model.add(Dense(24,activation="relu"))
# model.add(Dense(22,activation="relu"))

# #output layer
# model.add(Dense(1))

# #compile our model
# model.compile(loss="mean_absolute_error",optimizer="adam",metrics=["mae"])
# model.summary()

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

log_dir=r"regressionlogs"+ datetime.datetime.now().strftime("%y%m%d-%H%M%S")

tensorboard_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)
#set up Earlystopping
early_stopping=EarlyStopping(
    monitor="val_loss",
    patience=10,
    restore_best_weights=True
)

history = model.fit(
    x_train_Scaled,y_train,
    validation_data=(x_test_Scaled,y_test),
    epochs=50,
    callbacks=[early_stopping,tensorboard_callback]
    )

Epoch 1/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 100875.4375 - mae: 100875.4375 - val_loss: 98527.6094 - val_mae: 98527.6094
Epoch 2/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 99593.0469 - mae: 99593.0469 - val_loss: 97109.9297 - val_mae: 97109.9297
Epoch 3/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 99442.1953 - mae: 99442.1953 - val_loss: 93498.9062 - val_mae: 93498.9062
Epoch 4/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 94397.5781 - mae: 94397.5781 - val_loss: 87431.3516 - val_mae: 87431.3516
Epoch 5/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 87719.2344 - mae: 87719.2344 - val_loss: 79474.2734 - val_mae: 79474.2734
Epoch 6/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 79503.9844 - mae: 79503.9844 - val_loss: 70957.7031 - val_mae: 70957.7031
Epoch 7/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 69593.1875 - mae: 69593.1875 - val_loss: 63221.3398 - val_mae: 63221.3398
Epoch 8/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 